# Natural Language Processing

In [2]:
import os
import requests
import pandas as pd 
import numpy as np
import csv
import sys

csv.field_size_limit(sys.maxsize)

reviews = "/Users/skhederian/restaurant-health/format_reviews.csv"
data = "/Users/skhederian/restaurant-health/the_final_countdown.csv"

dfr = pd.read_csv(reviews)
dfd = pd.read_csv(data)

In [3]:
#Drop NaN Violations
dfreview = dfr.dropna(how = 'any').copy()
dfreview.shape

(20377, 5)

In [4]:
#Remove outliers that have high violation (> 4 standard deviations)
o = dfreview.copy()
odf = o[((o.violations - o.violations.mean()) / o.violations.std()).abs() < 4]
odf.shape

(20248, 5)

In [5]:
#Create bins for violation counts
bins = [-1, 5, 10, 15, 20, 25, 30, 35, 40]
group_names = ['Perfect', 'Excellent', 'Great', 'Good', 'Bad', 'Very Bad', 'rats', 'Shutdown']

odf['violationoutcat'] = pd.cut(odf['violations'], bins, labels=group_names)
odf.head()
odf.shape

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(20248, 6)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
counts = cv.fit_transform(odf['string_agg'].values)
counts

<20248x114008 sparse matrix of type '<class 'numpy.int64'>'
	with 7645234 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()

targetout = odf.violationoutcat.values
classifier.fit(counts, targetout)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [8]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  cv),
    ('classifier',  classifier)])

In [9]:
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


k_fold = KFold(n=len(odf), n_folds=12)
f1scores = []
ascores = []
rscores = []
pscores = []

for train_indices, test_indices in k_fold:
    train_text = odf.iloc[train_indices]['string_agg'].values
    train_y = odf.iloc[train_indices]['violationoutcat'].values

    test_text = odf.iloc[test_indices]['string_agg'].values
    test_y = odf.iloc[test_indices]['violationoutcat'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #scores
    f1score = f1_score(test_y, predictions, average='weighted')
    f1scores.append(f1score)
    accuracy = accuracy_score(test_y, predictions)
    ascores.append(accuracy)
    recall = recall_score(test_y, predictions, average='weighted')
    rscores.append(recall)
    precision = precision_score(test_y, predictions, average='weighted')
    pscores.append(precision)

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-sco

In [10]:
print('Total reviews classified:', len(odf))
print('F1 Score:', sum(f1scores)/len(f1scores))
print ('Accuracy:', sum(ascores)/len(ascores))
print ('Recall:', sum(rscores)/len(rscores))
print ('Precision:', sum(pscores)/len(pscores))

Total reviews classified: 20248
F1 Score: 0.494298051694
Accuracy: 0.521337724781
Recall: 0.521337724781
Precision: 0.481683076583


In [11]:
#Create a Pickle
from sklearn.externals import joblib

joblib.dump(pipeline, 'class.pkl', compress=9)

['class.pkl']

# Text Features

In [12]:
def most_informative_feature_for_class(cv, classifier, classlabel, n=30):
    labelid = list(classifier.classes_).index(classlabel)
    feature_names = cv.get_feature_names()
    topn = sorted(zip(classifier.coef_[labelid], feature_names)) [-n:]
    
    for coef, feat in topn:
        print (classlabel, feat, coef)

In [13]:
most_informative_feature_for_class(cv, classifier1, 'rats')

NameError: name 'classifier1' is not defined

In [ ]:
most_informative_feature_for_class(cv, classifier1, 'Perfect')

In [ ]:
cv.get_feature_names()

# Visualization

In [ ]:
from os import path
from scipy.misc import imread
import matplotlib.pyplot as plt
import random

from wordcloud import WordCloud, STOPWORDS

text = str(cv.get_feature_names())
wordcloud = WordCloud(font_path='/Library/Fonts/Verdana.ttf',
                      relative_scaling = 1.0,
                      stopwords = 'to of'
                      ).generate(text)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()